In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
a = torch.rand([32, 100, 3200])

In [ ]:
convs = nn.ModuleList(
            [
                nn.Conv2d(
                    1,
                    100,
                    (K, 600 * 2 * (4 if conf.expand_features else 1)),
                )
                for K in conf.filter_sizes
            ]
        )